In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
import tensorflow as tf
from tensorflow import keras

In [157]:
np.random.seed(42)

In [159]:
data=pd.read_csv('EURUSD_tick_OK-2.csv')

In [161]:
n_small=250000
data_a=np.array(data[:n_small])
N=50
L=n_small-N
data_b=np.zeros((L,N,5), dtype='float32')
Y=np.zeros((L,2),dtype='float32')

In [163]:
from tqdm import tqdm

In [165]:
for k in tqdm(range(n_small-N)):
    data_b[k,:,:]=data_a[k:k+N, :]
    Y[k,0]=data_a[k+N,3]
    Y[k,1]=data_a[k+N,1]

100%|██████████| 249950/249950 [00:01<00:00, 249413.75it/s]


In [167]:
train_start=0
train_end=int(np.floor(0.9*L))
train_end -=4955
test_start=train_end+1
test_end=L
X_train=np.zeros((train_end,N,5),dtype='float32')
X_test=np.zeros((test_end-train_end,N,5),dtype='float32')
y_train=Y[np.arange(train_start, train_end),:]
y_test=Y[np.arange(test_start,L),:]
data_add_row=np.array([y_test[L-test_start-1,0],y_test[L-test_start-1,1]])
y_test=np.vstack([y_test,data_add_row])

In [169]:

for z in tqdm(range(test_end)):
    if z<train_end:
        X_train[z,:,:]=data_b[z,:,:]
    else:
        X_test[z-train_end,:,:]=data_b[z,:,:]

100%|██████████| 249950/249950 [00:00<00:00, 429834.65it/s]


In [171]:
model=keras.Sequential([
    keras.layers.Conv1D(50, 5, padding='same',input_shape=(50, 5), activation=tf.nn.relu, kernel_initializer="normal"),
    keras.layers.MaxPooling1D(7),
    keras.layers.Conv1D(100, 5, padding='same', activation=tf.nn.relu, kernel_initializer="normal"),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(25,activation=tf.nn.relu,kernel_initializer="normal"),
    keras.layers.Dense(2)
])
print(model.summary())
    

D:\SHAMEEM\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_21 (Conv1D)              │ (None, 50, 50)         │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 7, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 7, 100)         │        25,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_7          │ (None, 100)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 25)             │         2,525 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │            52 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,977 (113.19 KB)

 Trainable params: 28,977 (113.19 KB)

 Non-trainable params: 0 (0.00 B)

None


In [173]:
model.compile(optimizer="adam",loss="mse",metrics=["mae"])

In [175]:
#student id  is 2370814
#so epochs is 1+4=5
#batch_size=50
history = model.fit(X_train, y_train, batch_size=50, epochs=5, validation_split=0.2, verbose=1)

Epoch 1/5
3520/3520 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.0081 - mae: 0.0445 - val_loss: 0.0011 - val_mae: 0.0232
Epoch 2/5
3520/3520 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 7.6732e-04 - mae: 0.0188 - val_loss: 8.5943e-04 - val_mae: 0.0191
Epoch 3/5
3520/3520 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 7.2610e-04 - mae: 0.0182 - val_loss: 8.5947e-04 - val_mae: 0.0190
Epoch 4/5
3520/3520 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 7.0874e-04 - mae: 0.0180 - val_loss: 0.0013 - val_mae: 0.0267
Epoch 5/5
3520/3520 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 7.0067e-04 - mae: 0.0177 - val_loss: 8.5180e-04 - val_mae: 0.0190


In [177]:
mse, mae=model.evaluate(X_test,y_test, verbose=1)
print("Mean absolute error: %.5f" %mae)

936/936 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0012 - mae: 0.0234
Mean absolute error: 0.02483


In [181]:
print("Mean absolute error in practical section : 0.02579")
print("Mean absolute error in logbook work      : %.5f" %mae)

Mean absolute error in practical section : 0.02579
Mean absolute error in logbook work      : 0.02483
